# Imports

In [32]:
# Importing the required packages
from tqdm import tqdm
import datetime
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import time, sys

# Data Files

In [56]:
# importing the data 
RF_Data = pd.read_csv("C:\\Users\\Ritvik Bale\\Documents\\Systemic Risk Research\\Risk Factor Data v1.csv")
Bank_Data = pd.read_csv("C:\\Users\\Ritvik Bale\\Documents\\Systemic Risk Research\\Bank data v1 2020-08-24.csv")

# converting the raw data to DataFrames
RF_Data = pd.DataFrame(RF_Data)
Bank_Data = pd.DataFrame(Bank_Data)

# Adding Year, Month, and Day Columns to the DataFrames

In [57]:
#Adding Year, Month, and Day columns to the BankData Dataframe 
#This makes it easier to access the specific rows that we want

#Initializing the arrays that will store the Year, Month, and Day of all the rows in the BankData DataFrame
Year = []
Month = []
Day = []

for i in Bank_Data["date"]:    
    #Getting the Year, Month, and Day for each row in BankData DataFrame
    Year.append((datetime.datetime.strptime(str(i), "%Y%m%d")).year)
    Month.append((datetime.datetime.strptime(str(i), "%Y%m%d")).month)
    Day.append((datetime.datetime.strptime(str(i), "%Y%m%d")).day)
    
#Adding the Year, Month, and Day arrays as columns to the BankData DataFrame
Bank_Data["Year"] = Year
Bank_Data["Month"] = Month
Bank_Data["Day"] = Day

In [58]:
#Adding Year, Month, and Day columns to the RiskFactor Dataframe 
#This makes it easier to access the specific rows that we want

#Initializing the arrays that will store the Year, Month, and Day of all the rows in the RiskFactor DataFrame
Year = []
Month = []
Day = []

for i in RF_Data["Date"]:    
    #Getting the Year, Month, and Day for each row in the RiskFactor DataFrame
    Year.append((datetime.datetime.strptime(i, "%Y-%m-%d")).year)
    Month.append((datetime.datetime.strptime(i, "%Y-%m-%d")).month)
    Day.append((datetime.datetime.strptime(i, "%Y-%m-%d")).day)
    
#Adding the Year, Month, and Day arrays as columns to the RiskFactor DataFrame
RF_Data["Year"] = Year
RF_Data["Month"] = Month
RF_Data["Day"] = Day

In [288]:
#First few rows of BankData DataFrame
Bank_Data.head()

,PERMNO,date,SICCD,COMNAM,PERMCO,CUSIP,RET,Year,Month,Day
0,10071,19920331,6211,ALEX BROWN INC,8015,1390210,-0.082353,1992,3,31
1,10071,19920430,6211,ALEX BROWN INC,8015,1390210,-0.174359,1992,4,30
2,10071,19920529,6211,ALEX BROWN INC,8015,1390210,0.125,1992,5,29
3,10071,19920630,6211,ALEX BROWN INC,8015,1390210,-0.0625,1992,6,30
4,10071,19920731,6211,ALEX BROWN INC,8015,1390210,-0.045926,1992,7,31


In [289]:
#First few rows of RiskFactor DataFrame
RF_Data.head()

,Date,BOND,CREDIT,SP500,CMDTY,DVIX,DHOUSE,Unnamed: 7,Year,Month,Day
0,1975-02-01,0.0388,-14.02,0.1386,-0.1654,-3.8780,0.0024,NaN,1975,2,1
1,1975-03-01,0.0157,-12.54,0.0702,-0.1036,-1.6952,0.0004,NaN,1975,3,1
2,1975-04-01,-0.0142,-12.71,-0.0014,0.0908,-0.3795,-0.0012,NaN,1975,4,1
3,1975-05-01,-0.0093,-12.02,0.0691,-0.0435,-1.0668,0.0004,NaN,1975,5,1
4,1975-06-01,0.0120,-12.81,0.0538,-0.0270,-0.9265,0.0008,NaN,1975,6,1


# Initializing the Return Table

In [312]:
#Initializing the Return_Table that will consist of all the Bank Returns, and the corresponding Risk Factor Returns
Return_Table = []
Return_Table = pd.DataFrame(Return_Table)

#Adding Columns for the Year, Month, and Day (allows us to easily retrieve the corresponding Risk Factor Returns (by date))
Return_Table["Year"] = Bank_Data["Year"]
Return_Table["Month"] = Bank_Data["Month"]
Return_Table["Day"] = Bank_Data["Day"]

#Adding a PERMNO column(to be able to identify the firms), and a column consisting of all the Bank Returns
Return_Table["PERMNO"] = Bank_Data["PERMNO"]
Return_Table["Return"] = Bank_Data["RET"]

### Adding the Risk Factor Returns to the Return Table

In [313]:
#Initializing a Return Array for each Risk Factor
BOND = []
CREDIT = []
SP500 = []
CMDTY = []
DVIX = []
DHOUSE = []

In [314]:
#Adding all the corresponding Risk Factor Return values for each Bank Return entry (by date)
#Looping through the Bank Data DataFrame to find the Risk Factor returns for each date
for index, i in Bank_Data.iterrows():
    
    #Determining the year and month of row
    year = RF_Data["Year"] == i["Year"]
    month = RF_Data["Month"] == i["Month"]
    
    #Determining the row of Risk Factor Returns for the given date (year & month)
    Value = RF_Data[year & month]
    
    #Appending the Risk Factor Returns for the given date (year & month)
    BOND.append(np.array(Value["BOND"]))
    CREDIT.append(np.array(Value["CREDIT"]))
    SP500.append(np.array(Value["SP500"]))
    CMDTY.append(np.array(Value["CMDTY"]))
    DVIX.append(np.array(Value["DVIX"]))
    DHOUSE.append(np.array(Value["DHOUSE"]))

In [315]:
#Adding all of the Corresponding Risk Factor Return Arrays to the DataFrame
Return_Table["BOND"] = BOND
Return_Table["CREDIT"] = CREDIT
Return_Table["SP500"] = SP500
Return_Table["CMDTY"] = CMDTY
Return_Table["DVIX"] = DVIX
Return_Table["DHOUSE"] = DHOUSE

In [316]:
#First few rows of ReturnTable DataFrame
Return_Table.head()

,Year,Month,Day,PERMNO,Return,BOND,CREDIT,SP500,CMDTY,DVIX,DHOUSE
0,1992,3,31,10071,-0.082353,[-0.0015],[-3.676],[0.0106],[-0.0237],[-0.5],[0.0016]
1,1992,4,30,10071,-0.174359,[-0.0124],[-3.344],[-0.0178],[0.0306],[-0.65],[0.0009]
2,1992,5,29,10071,0.125,[-0.0006],[-3.179],[0.022],[0.0231],[-1.67],[0.0007]
3,1992,6,30,10071,-0.0625,[0.0066],[-3.319],[0.0159],[0.0243],[-0.51],[-0.0008]
4,1992,7,31,10071,-0.045926,[0.0078],[-3.212],[-0.0084],[-0.0041],[-0.18],[-0.0008]


# Coefficient (Beta) DataFrame Creation

In [317]:
#Creating an array consisting of all the PERMNO Codes (each code is only counted once - unique)
PERMNO = Bank_Data["PERMNO"].unique()

#Creating the Coefficient Table DataFrame which will contain the coefficients from all of the regressions
Coefficient_Table = []
Coefficient_Table = pd.DataFrame(Coefficient_Table, columns = ["BOND", "CREDIT", "SP500", "CMDTY", "DVIX", "DHOUSE"])

In [318]:
#Initial Columns of the Coefficient Table
Coefficient_Table

,BOND,CREDIT,SP500,CMDTY,DVIX,DHOUSE


In [319]:
def DateFunction(start_year, end_year, start_month, end_month):
    
    #Iterate over all firms
    for i in PERMNO:
        #Create a Return table for each firm - consisting of only the dates that have been specified
        Single_Firm = Return_Table[i]
        
        #Setting the start and end, year and month variables - to be unputtted into the function which retrieves only the rows corresponding to specified dates
        start_y = Single_Firm["Year"] >= start_year
        end_y = Single_Firm["Year"] <= end_year
        start_m = Single_Firm["Month"] >= start_month
        end_m = Single_Firm["Month"] <= end_month
        
        #Modifying the DataFrame to only include returns from the specified dates
        SF = Single_Firm[start_y & end_y & start_m & end_m]
        
        #Initializing the Linear Regression Function
        linear_regression = LinearRegression()
        
        #Setting the X Variables to be regressed on (Risk Factors), and setting the Y Variable that will be regressed (Bank Returns)
        y = Single_Firm['Return']
        x = Single_Firm[['BOND'], ['CREDIT'], ['SP500'], ['CMDTY'], ['DVIX'], ['DHOUSE']]
        
        #Conducting the Linear Regression
        linear_regression.fit(x, y)
        
        #Converting the set of coefficients to an Array and appending it to the Coefficient DataFrame
        Coefficients = np.array(linear_regression.coef_)
        Coefficient_Table.append(Coefficients)


In [1]:
#Initialzing the Distance Table - will contain the distances of all the firms from the line of Complete Systemic Risk
Distance_Table = []

In [320]:
Single_Firm


,Year,Month,Day,PERMNO,Return,BOND,CREDIT,SP500,CMDTY,DVIX,DHOUSE
0,1992,3,31,10071,-0.082353,[-0.0015],[-3.676],[0.0106],[-0.0237],[-0.5],[0.0016]
1,1992,4,30,10071,-0.174359,[-0.0124],[-3.344],[-0.0178],[0.0306],[-0.65],[0.0009]
2,1992,5,29,10071,0.125,[-0.0006],[-3.179],[0.022],[0.0231],[-1.67],[0.0007]
3,1992,6,30,10071,-0.0625,[0.0066],[-3.319],[0.0159],[0.0243],[-0.51],[-0.0008]


# Statistical Analysis

## Running PCA to Determine the Level of Correlation

In [ ]:
#Initializing Standardization Function
sc = StandardScaler()

#Initializing a Standardized Matrix of the Coefficient Table
Coeff_PCA_std = sd.fit_transform(Coefficients_Table)

#Condstructing a Covariance Matrix from the Standardized Matrix
Cov_Mat = np.cov(Coeff_PCA_std.T)

#Determining the Eigenvectors and the Eigenvalues from the Covariance Matrix
Eigen_Vals, Eigen_Vecs = np.linalg.eig(Cov_Mat)

#Determining the Percentage Explained Variances
Total = sum(Eigen_Vals)
Var_Exp = [(i / Total) for i in sorted(Eigen_Vals, reverse=True)]

## Determining the Distance of the Points from the Line of Complete Systemic Risk

In [ ]:
#Creating an Array which wil containg the average of the coefficient of all the points (the mean coordinates)
Average_Coeff = []

#Appending the Average Beta of each Risk Factor to the Average_Coeff Array
Average_Coeff.append[np.mean(np.array(Coefficient_Table['BOND']))]
Average_Coeff.append[np.mean(np.array(Coefficient_Table['SP500']))]
Average_Coeff.append[np.mean(np.array(Coefficient_Table['CREDIT']))]
Average_Coeff.append[np.mean(np.array(Coefficient_Table['CMDTY']))]
Average_Coeff.append[np.mean(np.array(Coefficient_Table['DVIX']))]
Average_Coeff.append[np.mean(np.array(Coefficient_Table['DHOUSE']))]

In [ ]:
#Initializing the Normal(Perpendicular/Orthogonal) Vector to our Surface/Manifold of Complete Systemic Risk
Normal = [1, 1, 1, 1, 1, 1]

#Initializing the Vector between a Randomly Selected Point on the Manifold (1, 0, 0, 0, 0, 0) and the Average_Coeff Point
Random_Point = [1, 0, 0, 0, 0, 0]
Vector = np.subtract(Average_Coeff, Random_Point)

#Taking the Dot Product of the Normal Vector and the Arbitrary Vector to determine the distance of the point from the Manifold of Complete Systemic Risk
Distance = np.dot(Normal, Vector)

## Squared Weighted Systemic Risk

In [ ]:
#Calculating the Squared Weighted Systemic Risk

#Initializing the Square Weighted Systemic Risk Variable
SquareWeighted_Systemic_Risk = 0

#Looping through the Coefficients and Incrementing the Variable by each Squared Coefficient
for i in Average_Coeff:
    #Incrementing by the Squared Coefficient
    SquareWeighted_Systemic_Risk += (i**2)
    

## Measuring Clustering via the SD of the Points from the Mean Point

In [ ]:
#Initializing the Array which will hold the Distances of all the Points from the Mean Point
distances = []

#Iterating through all Firms and their Coefficients in the Coefficients Table
for i in Coefficient_Table:
    #Initialzing the Squared Differences for each Coefficient
    Bond_Distance = (i["BOND"] - Average_Coeff[0])**2
    SP500_Distance = (i["SP500"] - Average_Coeff[1])**2
    Credit_Distance = (i["CREDIT"] - Average_Coeff[2])**2
    CMDTY_Distance = (i["CMDTY"] - Average_Coeff[3])**2
    DVIX_Distance = (i["DVIX"] - Average_Coeff[4])**2
    DHouse_Distance = (i["DHOUSE"] - Average_Coeff[5])**2
    
    #Determining the Distance via all Coefficient Differences and the Pythagorean Theorem
    distance = sqrt((Bond_Distance + SP500_Distance + Credit_Distance + CMDTY_Distance + DVIX_Distance + DHouse_Distance))
    
    #Appending the distance to the distances array
    distances.append(distance)

#Calculating the SD of the Distances array (the smaller the SD the greater the Clustering and Systemic Risk)
sd_distances = np.std(distances)

## Running K-Means Clustering

In [ ]:
#Initializing the SSE Array Which Will Contain the Sum of Squared Errors for each Number of Clusters
SSE = []

#Running K-Means Clustering from 1 - 11 Clusters, and appending the SSE for each Number of Clusters
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(Coefficients_Table)
    wcss.append(kmeans.inertia_)

#Creating the Elbow Plot showing the SSE for each Number of Clusters as a Contiguous Line
plt.plot(range(1, 11), SSE)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()